In [1]:
%reset -sf

In [7]:
import torch
from torch.utils.checkpoint import checkpoint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
from opt_einsum import contract
from torch.profiler import profile, record_function, ProfilerActivity
def _toN(t):
    return t.detach().cpu().numpy()
def printDiff(name,value,ref):
    if ref is not None:
        print(name+':',value,'diff(abs):',value-ref)
    else:
        print(name+':',value)
    
import ast
def eval_np_array_literal(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

torch.set_default_tensor_type(torch.cuda.DoubleTensor)

In [8]:
# SRG BaseClass, can be used for HOTRG-like and XTRG-like contraction
class SRG(torch.nn.Module):
    def __init__(self,params,options):
        super(SRG,self).__init__()
        self.dtype={'float64':torch.float64,'float32':torch.float32}[options.get('dtype','float64')]
        self.device=options.get('device','cpu')
        self.max_dim=options.get('max_dim',16)
        self.nLayers=options.get('nLayers',20)
        self.use_checkpoint=options.get('use_checkpoint',True)
        self.observable_checkerboard=False
        self.use_bond_symmetry=False
        
        self.params=torch.nn.ParameterDict({
            k:torch.nn.Parameter(torch.tensor(v,dtype=self.dtype,device=self.device)) for k,v in params.items()
        })
        self.persistent={}
        self.persistent['logZ']=0
        
    def __str__(self):
        rtval=""
        for k,v in self.params.items():
            rtval+=k+':'+v+'\n'
        rtval+='dtype:'+self.dtype+'\n'
        rtval+='device:'+self.device+'\n'
        rtval+='max_dim:'+self.max_dim+'\n'
        rtval+='nLayers:'+self.nLayers+'\n'
        rtval+='nSite:'+2.0**nLayers+'\n'
        
    def set_params(self,params):
        self.params=torch.nn.ParameterDict({
            k:torch.nn.Parameter(torch.tensor(v,dtype=self.dtype,device=self.device)) for k,v in params.items()
        })
        
    def toT(self,t):
        return torch.tensor(t,dtype=self.dtype,device=self.device)
    
    def generate_random_Isometry(self,dim1,dim2):
        dim=max(dim1,dim2)
        A=torch.randn(dim,dim,dtype=self.dtype,device=self.device)
        U=torch.matrix_exp(A-A.t())
        U=U[:dim1,:dim2]
        return U
    
    def TRG_same_T(self,T,*w):
        return self.TRG(T,T,*w)
    
    def _checkpoint(self,F,*ww):
        requires_grad=False
        for w in ww:
            if w.requires_grad:
                requires_grad=True
        if self.use_checkpoint and requires_grad:
            return torch.utils.checkpoint.checkpoint(F,*ww)
        else:
            return F(*ww)
    
    def forward_tensor(self,contract_method=None):
        T=self.get_T0()
        logTotal=0
        contracted=torch.zeros((self.nLayers,2),dtype=self.dtype,device=self.device)
        for i in range(self.nLayers):
            w=self.ws[(i*self.w_per_layer):((i+1)*self.w_per_layer)]
            T=self._checkpoint(self.TRG_same_T,T,*w)
                
            norm=torch.linalg.norm(T)
            T=T/norm
            logTotal=2*logTotal+torch.log(norm)
            
            if contract_method is not None:
                Z=contract(T,contract_method)
                contracted[i,0]=Z
                contracted[i,1]=norm
        return T,logTotal,contracted
    
    def forward_tensor_with_observable(self,T_op,contract_method=None,start_layer=0):
        T=self.get_T0()
        for i in range(start_layer):
            w=self.ws[(i*self.w_per_layer):((i+1)*self.w_per_layer)]
            T=self._checkpoint(self.TRG_same_T,T,*w)
            
        logTotal=0
        contracted=torch.zeros((self.nLayers,3),dtype=self.dtype,device=self.device)
        for i in range(start_layer,self.nLayers):
            w=self.ws[(i*self.w_per_layer):((i+1)*self.w_per_layer)]
            T1=self._checkpoint(self.TRG_same_T,T,*w)
            T2=self._checkpoint(self.TRG,T,T_op,*w)
            T3=self._checkpoint(self.TRG,T_op,T,*w)
            if self.observable_checkerboard and i<self.spacial_dim:
                T3=-T3

            T,T_op=T1,(T2+T3)/2
            norm=torch.linalg.norm(T)
            T,T_op=T/norm,T_op/norm
            logTotal=2*logTotal+torch.log(norm)
            
            if contract_method is not None:
                Z=contract(T,contract_method)
                Z_op=contract(T_op,contract_method)
                contracted[i,0]=Z
                contracted[i,1]=Z_op
                contracted[i,2]=norm
            
        return T,T_op,logTotal,contracted
    
    
    def dlogZ(self,param):
        self.requires_grad_(False)
        self.params[param].requires_grad_(True)
        self.zero_grad()
        logZ=self.forward()
        logZ.backward()
        result=_toN(self.params[param].grad)
        self.params[param].requires_grad_(False)
        return result
    
    def update_single_layer(self,layer):
        self.requires_grad_(False)
        
        for i in range(layer*self.w_per_layer,(layer+1)*self.w_per_layer):
            self.ws[i].requires_grad_(True)
        self.zero_grad()
        
        logZ=self.forward()
        logZ.backward()
        
        with torch.no_grad():
            for i in range(layer*self.w_per_layer,(layer+1)*self.w_per_layer):
                E=self.ws[i].grad
                dim1,dim2=E.shape[0],E.shape[2]
                E=E.reshape(dim1*dim1,dim2)
                U,S,Vh=torch.linalg.svd(E,full_matrices=False)
                UVh=U@Vh
                #UVh=svd2UVh(E)
                del U,S,Vh,E
                
                #calculate diff
                UVh_old=self.ws[i].reshape(dim1*dim1,dim2)
                self.persistent['ws_diff'][i]=_toN(torch.norm(UVh_old.t()@UVh@UVh.t()@UVh_old-torch.eye(dim2,device=UVh.device)))
                del UVh_old
                    
                self.ws[i].data=UVh.reshape(dim1,dim1,dim2)
                del UVh
                torch.cuda.empty_cache()
        return _toN(logZ)
        
    def optimize(self,nIter):
        self.persistent['ws_diff']=np.zeros(len(self.ws))
        
        torch.cuda.empty_cache()
        if nIter>1:
            pbar2=tqdm(range(nIter), leave=False)
            pbar2.set_postfix({k:_toN(v) for k,v in self.params.items()})
        else:
            pbar2=range(nIter)
        for i in pbar2:
            pbar=tqdm([*range(self.nLayers-1,-1,-1)]+[*range(self.nLayers)], leave=False)
            for j in pbar:
                ws_shape=self.ws[j*self.w_per_layer].shape
                if ws_shape[0]**2>ws_shape[2]:
                    logZ=self.update_single_layer(j)
                #else:
                #    print(f'Skip layer {j} shape={ws_shape}')
        #lock all grads
        for param in self.params.values(): 
            param.requires_grad_(False)
        for i in range(self.nLayers): #slightly faster
            self.ws[i].requires_grad_(False)
        
        self.persistent['logZ_diff']=np.abs(self.persistent['logZ']-logZ)
        self.persistent['logZ']=logZ
        
        # normalized by layer weight, number of elements in tensor
        # NOT USED but multiply by number of elements in last tensor to better match the effects in output
        self.persistent['ws_diff_normalized']=np.zeros(len(self.ws))
        for i in range(self.nLayers):
            for j in range(self.w_per_layer):
                ij=i*self.w_per_layer+j
                self.persistent['ws_diff_normalized'][ij]=self.persistent['ws_diff'][ij]/2.0**i#/torch.numel(self.ws[ij])*torch.numel(self.ws[-1])
        # ignore the last layers we take trace directly
        # use 10-norm so layers of large error has beter contribution       
        self.persistent['ws_diff_total']=np.average(self.persistent['ws_diff_normalized'][:-self.w_per_layer*self.spacial_dim])


In [9]:
class HOTRG(SRG):
    def __init__(self,params,options):
        super(HOTRG,self).__init__(params,options)
        self.nLayers_HOSVD=options.get('nLayers_HOSVD',0)
        self.persistent['magnetizationX']=0
        self.persistent['magnetizationY']=0
        self.persistent['magnetizationZ']=0
        self.persistent['energy']=0
    
    def create_isometries(self,start_dim,spacial_dim):
        ws=[]
        bond_dim=[start_dim]*spacial_dim
        for i in range(self.nLayers):
            for j in range(1,spacial_dim):
                old_dim=bond_dim[j]
                new_dim=min(old_dim**2,self.max_dim)
                U=self.generate_random_Isometry(old_dim**2,new_dim).view(old_dim,old_dim,new_dim)
                ws.append(U.detach())
                bond_dim[j]=new_dim
            bond_dim=bond_dim[1:]+[bond_dim[0]]
        self.ws=torch.nn.ParameterList([
            torch.nn.Parameter(v) for v in ws
        ])
        self.w_per_layer=spacial_dim-1
        self.spacial_dim=spacial_dim
        self.TRG={2:self.HOTRG2D,3:self.HOTRG3D}[self.spacial_dim]
        self.HOSVD={2:self.HOSVD2D,3:self.HOSVD3D}[self.spacial_dim]
        
        
    def HOTRG2D(self,T1,T2,w):
        return contract('ijkl,jmno,kna,lob->abim',T1,T2,w,w.conj())#contract and rotate
    
    def HOTRG3D(self,T1,T2,w1,w2):
        return contract('ijklmn,jopqrs,kpa,lqb,mrc,nsd->abcdio',T1,T2,w1,w1.conj(),w2,w2.conj())#contract and rotate
    
    def HOSVD2D(self,T1,T2,BS=None):
        MM1=contract('ijkl,jmno,ipql,pmro->knqr',T1,T2,T1.conj(),T2.conj()).reshape(T1.shape[2]*T2.shape[2],-1)
        S1,U1=torch.linalg.eigh(MM1) #S1 ascending U S Uh=MM
        eps1=torch.sum(torch.abs(S1[-self.max_dim:])) # might be slightly minus due to numerical error
        
        MM2=contract('ijkl,jmno,ipql,pmro->knqr',T1.transpose(2,3),T2.transpose(2,3),T1.conj().transpose(2,3),T2.conj().transpose(2,3)).reshape(T1.shape[3]*T2.shape[3],-1)
        S2,U2=torch.linalg.eigh(MM2)
        eps2=torch.sum(torch.abs(S2[-self.max_dim:]))

        S,U=(S1,U1) if eps1<eps2 else (S2,U2)
        w=U[:,-self.max_dim:].reshape(T1.shape[2],T2.shape[2],-1)
        
        T=contract('ijkl,jmno,kna,lob->abim',T1,T2,w,w.conj())
        
        assert (BS is not None) == self.use_bond_symmetry
        if BS is not None:
            Tsum=torch.zeros_like(T)
            #wsum=torch.zeros_like(w)
            for i in range(len(BS)):
                BS[i]=[
                    self.fix_unitary(contract('kl,no,kna,lob->ab',BS[i][1],BS[i][1],w,w.conj())),
                    BS[i][0]
                ]
                Tsum+=contract('ijkl,Ii,Jj,Kk,Ll->IJKL',T,BS[i][0],BS[i][0].conj(),BS[i][1],BS[i][1].conj())
                #wsum+=contract('kna,Aa->knA',w,BS[i][0])
            #w=wsum/len(BS) #seems wrong
            T=Tsum/len(BS)
        
        return T,[w],BS
    
    def HOSVD3D(self,T1,T2,BS=None):
        
        #if BS is not None:
        #    T1sum=torch.zeros_like(T1)
        #    T2sum=torch.zeros_like(T2)
        #    for i in range(len(BS)):
        #        T1sum+=contract('ijklmn,Ii,Jj,Kk,Ll,Mm,Nn->IJKLMN',T1,BS[i][0],BS[i][0].conj(),BS[i][1],BS[i][1].conj(),BS[i][2],BS[i][2].conj())
        #      
        #        T2sum+=contract('ijklmn,Ii,Jj,Kk,Ll,Mm,Nn->IJKLMN',T2,BS[i][0],BS[i][0].conj(),BS[i][1],BS[i][1].conj(),BS[i][2],BS[i][2].conj())
        #      
        #    T1=T1sum/len(BS)  
        #    T2=T2sum/len(BS)  
        
        #print(T1.shape)
        MM1=contract('ijklmn,jopqrs,itulmn,tovqrs->kpuv',T1,T2,T1.conj(),T2.conj()).reshape(T1.shape[2]*T2.shape[2],-1)
        S1,U1=torch.linalg.eigh(MM1) #S ascending U S Uh=MM
        eps1=torch.sum(torch.abs(S1[-self.max_dim:])) # might be slightly minus due to numerical error
        
        MM2=contract('ijklmn,jopqrs,itulmn,tovqrs->kpuv',T1.transpose(2,3),T2.transpose(2,3),T1.conj().transpose(2,3),T2.conj().transpose(2,3)).reshape(T1.shape[3]*T2.shape[3],-1)
        S2,U2=torch.linalg.eigh(MM2)
        eps2=torch.sum(torch.abs(S2[-self.max_dim:]))

        S,U=(S1,U1) if eps1<eps2 else (S2,U2)
        w1=U[:,-self.max_dim:].reshape(T1.shape[2],T2.shape[2],-1)
        
        MM1=contract('ijklmn,jopqrs,itklun,topqvs->mruv',T1,T2,T1.conj(),T2.conj()).reshape(T1.shape[4]*T2.shape[4],-1)
        S1,U1=torch.linalg.eigh(MM1) #S ascending U S Uh=MM
        eps1=torch.sum(torch.abs(S1[-self.max_dim:])) # might be slightly minus due to numerical error
        
        MM2=contract('ijklmn,jopqrs,itklun,topqvs->mruv',T1.transpose(4,5),T2.transpose(4,5),T1.conj().transpose(4,5),T2.conj().transpose(4,5)).reshape(T1.shape[5]*T2.shape[5],-1)
        S2,U2=torch.linalg.eigh(MM2)
        eps2=torch.sum(torch.abs(S2[-self.max_dim:]))

        S,U=(S1,U1) if eps1<eps2 else (S2,U2)
        w2=U[:,-self.max_dim:].reshape(T1.shape[4],T2.shape[4],-1)
        #print(w1.shape,w2.shape)
        
        #T=contract('ijklmn,jopqrs,kpa,lqb,mrc,nsd->abcdio',T1,T2,w1,w1.conj(),w2,w2.conj())
        
        if BS is not None:
            Tsum=torch.zeros_like(T)
            #w1sum,w2sum=torch.zeros_like(w1),torch.zeros_like(w2)
            for i in range(len(BS)):
                BS[i]=[
                    self.fix_unitary(contract('kl,pq,kpa,lqb->ab',BS[i][1],BS[i][1],w1,w1.conj())),
                    self.fix_unitary(contract('mn,rs,mrc,nsd->cd',BS[i][2],BS[i][2],w2,w2.conj())),
                    BS[i][0]
                ]
                Tsum+=contract('ijklmn,Ii,Jj,Kk,Ll,Mm,Nn->IJKLMN',T,BS[i][0],BS[i][0].conj(),BS[i][1],BS[i][1].conj(),BS[i][2],BS[i][2].conj())
                #w1sum+=contract('kpa,Aa->kpA',w1,BS[i][0])
                #w2sum+=contract('mrc,Cc->mrC',w2,BS[i][1])
            T=Tsum/len(BS)
            #w1,w2=w1sum/len(BS),w2sum/len(BS) #seems wrong
        T=contract('ijklmn,jopqrs,kpa,lqb,mrc,nsd->abcdio',T1,T2,w1,w1.conj(),w2,w2.conj())
        return T,[w1,w2],BS
    
    def fix_unitary(self,M):
        U,S,Vh=torch.linalg.svd(M)
        S/=torch.abs(S)
        return U*S@Vh
    
    def fix_projector(self,P):
        S,U=torch.linalg.eigh(P) #S ascending U S Uh=P
        S[S<.5]=0
        S[S>.5]=1
        return U*S@(U.T.conj())
    
    def generate_isometries_HOSVD(self):
        with torch.no_grad():
            logTotal=0
            T=self.get_T0()
            BS=self.get_bond_symmetries() if self.use_bond_symmetry else None
            for i in tqdm(range(self.nLayers), leave=False):
                T,ww,BS=self.HOSVD(T,T,BS)
                for j in range(self.w_per_layer):
                    self.ws[i*self.w_per_layer+j].data=ww[j]
                norm=torch.linalg.norm(T)
                T=T/norm
                logTotal=2*logTotal+torch.log(norm)

            contract_all=[i for i in range(len(T.shape)//2) for j in range(2)]
            Z=contract(T,contract_all)
            self.persistent['logZ']=_toN((torch.log(Z)+logTotal)/2.0**self.nLayers)
    
    def forward(self):
        contract_method=[i for i in range(self.spacial_dim) for j in range(2)]
        T,logTotal,_=self.forward_tensor(contract_method=None)
        #contract_all=[i for i in range(len(T.shape)//2) for j in range(2)]
        Z=contract(T,contract_method)
        return (torch.log(torch.abs(Z))+logTotal)/2.0**self.nLayers
    
    def forward_and_HOTRG(self):
        contract_method=[i for i in range(self.spacial_dim) for j in range(2)]
        T,logTotal,contracted=self.forward_tensor(contract_method=contract_method)
        for i in range(self.nLayers_HOSVD):
            assert False # BS not cached
            T,_=self.HOSVD(T,T)
            norm=torch.linalg.norm(T)
            T=T/norm
            logTotal=2*logTotal+torch.log(norm)
        Z=contract(T,contract_method)
        return (torch.log(torch.abs(Z))+logTotal)/2.0**(self.nLayers+self.nLayers_HOSVD),contracted
    
    def forward_with_observable_and_HOTRG(self,T_op,start_layer=0):
        contract_method=[i for i in range(self.spacial_dim) for j in range(2)]
        T,T_op,logTotal,contracted=self.forward_tensor_with_observable(T_op,contract_method=contract_method,start_layer=start_layer)
        
        for i in range(self.nLayers_HOSVD):
            assert False # BS not cached
            T1,ws=self.HOSVD(T,T)
            T2=self.TRG(T,T_op,*ws)
            T3=self.TRG(T_op,T,*ws)
            
            T,T_op=T1,(T2+T3)/2
            norm=torch.linalg.norm(T)
            T,T_op=T/norm,T_op/norm
            logTotal=2*logTotal+torch.log(norm)
            
        Z=contract(T,contract_method)
        Z_op=contract(T_op,contract_method)
        
        return Z_op/Z,contracted
    
    def calc_logZ(self):
        with torch.no_grad():
            a,b=self.forward_and_HOTRG()
            self.persistent['logZ_contracted_per_layer']=_toN(b)
            logZ=_toN(a)
            self.persistent['logZ_diff']=np.abs(self.persistent['logZ']-logZ)
            self.persistent['logZ']=logZ
    
    def calc_magnetization(self):
        axes=[0,1,2] if hasattr(self,'get_ST0') else [2]
        for axis in axes:
            with torch.no_grad():
                axisLabel=['X','Y','Z'][axis]
                ST0=self.get_ST0(axis) if hasattr(self,'get_ST0') else self.get_SZT0()
                a,b=self.forward_with_observable_and_HOTRG(ST0)
                self.persistent['magnetization'+axisLabel+'_contracted_per_layer']=_toN(b)
                magnetization=_toN(torch.abs(a))
                self.persistent['magnetization'+axisLabel+'_diff']=np.abs(self.persistent['magnetization'+axisLabel]-magnetization)
                self.persistent['magnetization'+axisLabel]=magnetization
            
    def calc_energy(self):
        with torch.no_grad():
            a,b=self.forward_with_observable_and_HOTRG(self.get_ET1(),start_layer=1)
            self.persistent['energy_contracted_per_layer']=_toN(b)
            energy=_toN(a)
            self.persistent['energy_diff']=np.abs(self.persistent['energy']-energy)
            self.persistent['energy']=energy
            
    def calc_all(self):
        self.calc_logZ()
        if hasattr(self,'get_ST0') or hasattr(self,'get_SZT0'):
            self.calc_magnetization()
        if hasattr(self,'get_ET1'):
            self.calc_energy()

In [10]:
from scipy.special import comb
def get_CG_no_normalization(j):
    n=int(2*j)
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    return CG
def get_CG(j):
    n=int(2*j)
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1/np.sqrt(comb(n,m))
    return CG
def get_Singlet():
    return np.array([[0,1.],[-1.,0]])
def get_Lxyz(j):
    n=int(2*j+1)
    Lz=np.zeros((n,n))
    for i in range(n):
        m=i-j
        Lz[i,i]=m
    Lp=np.zeros((n,n))
    for i in range(n-1):
        m=i-j
        Lp[i+1,i]=np.sqrt(j*(j+1)-m*(m+1))
    Lm=Lp.T
    Lx=(Lp+Lm)/2
    iLy=(Lp-Lm)/2
    return Lx,iLy,Lz

class AKLT2D(HOTRG):
    default_params={'a1':np.sqrt(6)/2,'a2':np.sqrt(6)}
    def __init__(self,params,options):
        super(AKLT2D,self).__init__(params,options)
        self.create_isometries(start_dim=4,spacial_dim=2)
        self.observable_checkerboard=True
        self.use_bond_symmetry=True
        
    def get_T0(self):
        projector=self.toT(get_CG_no_normalization(2))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2=self.toT(1),self.params['a1'],self.params['a2']
        deform=torch.stack([ac2,ac1,ac0,ac1,ac2])
        node=contract('aijkl,im,kn,a->amjnl',projector,singlet,singlet,deform)
        return contract('aijkl,amnop->imjnkolp',node,node).reshape(4,4,4,4)#UDLR

    def get_ST0(self,axis):
        projector=self.toT(get_CG_no_normalization(2))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2=self.toT(1),self.params['a1'],self.params['a2']
        deform=torch.stack([ac2,ac1,ac0,ac1,ac2])
        node=contract('aijkl,im,kn,a->amjnl',projector,singlet,singlet,deform)
        op=self.toT(get_Lxyz(2)[axis])
        return contract('aijkl,bmnop,ab->imjnkolp',node,node,op).reshape(4,4,4,4)#UDLR
    
    def get_bond_symmetries(self):
        Id=self.toT(np.eye(4))
        Swap=contract('ijkl->ijlk',Id.reshape(2,2,2,2)).reshape(4,4)
        return [[Id]*self.spacial_dim,[Swap]*self.spacial_dim]
    
    

class AKLT3D(HOTRG):
    default_params={'a1':np.sqrt(20/15),'a2':np.sqrt(20/6),'a3':np.sqrt(20/1)}
    def __init__(self,params,options):
        super(AKLT3D,self).__init__(params,options)
        self.create_isometries(start_dim=4,spacial_dim=3)
        self.observable_checkerboard=True
        self.use_bond_symmetry=True
        
    def get_T0(self):
        projector=self.toT(get_CG_no_normalization(3))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2,ac3=self.toT(1),self.params['a1'],self.params['a2'],self.params['a3']
        deform=torch.stack([ac3,ac2,ac1,ac0,ac1,ac2,ac3])
        node=contract('aijklmn,io,kp,mq,a->aojplqn',projector,singlet,singlet,singlet,deform)
        return contract('aijklmn,aopqrst->iojpkqlrmsnt',node,node).reshape(4,4,4,4,4,4)#UDLRFB

    def get_ST0(self,axis):
        projector=self.toT(get_CG_no_normalization(3))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2,ac3=self.toT(1),self.params['a1'],self.params['a2'],self.params['a3']
        deform=torch.stack([ac3,ac2,ac1,ac0,ac1,ac2,ac3])
        node=contract('aijklmn,io,kp,mq,a->aojplqn',projector,singlet,singlet,singlet,deform)
        op=self.toT(get_Lxyz(3)[axis])
        return contract('aijklmn,bopqrst,ab->iojpkqlrmsnt',node,node,op).reshape(4,4,4,4,4,4)#UDLRFB
    
    def get_bond_symmetries(self):
        Id=self.toT(np.eye(4))
        Swap=contract('ijkl->ijlk',Id.reshape(2,2,2,2)).reshape(4,4)
        return [[Id]*self.spacial_dim,[Swap]*self.spacial_dim]

In [19]:
options={'nLayers':15,'max_dim':10,'device':'cuda:0'}
params=AKLT3D.default_params.copy()
params['a1']+=.008

aklt=AKLT3D(params,options)
aklt.use_bond_symmetry=False
aklt.generate_isometries_HOSVD()


aklt.calc_all()

  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
aklt.persistent

{'logZ': array(3.09407501),
 'magnetizationX': array(3.97433239e-13),
 'magnetizationY': array(2.29447344e-16),
 'magnetizationZ': array(1.19859808e-16),
 'energy': 0,
 'logZ_contracted_per_layer': array([[ 9.88481293e-02,  1.50571933e+03],
        [-3.57110611e-01,  3.44685473e-01],
        [ 2.46549214e+00,  3.17632194e-01],
        [ 2.64152437e+00,  2.98910819e-01],
        [ 2.75367040e+00,  2.80430852e-01],
        [ 2.87228393e+00,  2.81057899e-01],
        [ 2.93547720e+00,  2.67168063e-01],
        [ 2.89630102e+00,  2.88754572e-01],
        [ 2.98307306e+00,  2.97062677e-01],
        [ 3.04846254e+00,  2.75067481e-01],
        [ 2.47188026e+00,  3.27431947e-01],
        [ 2.15631284e+00,  3.75473048e-01],
        [ 1.88005917e+00,  3.96033690e-01],
        [ 1.84161170e+00,  3.96940167e-01],
        [ 1.81297051e+00,  3.73375485e-01]]),
 'logZ_diff': 0.0,
 'magnetizationX_contracted_per_layer': array([[ 9.88481293e-02,  9.86076132e-32,  1.50571933e+03],
        [-3.57110611e-

In [5]:
options={
    'dtype':'float64',
    'device':'cuda:0',
    'max_dim':10, # 10 discussed with wei
    'nLayers':15, # 30
    'use_checkpoint':True
}
params=AKLT3D.default_params.copy()
params['a1']+=.008
model=AKLT3D(params,options)
model.generate_isometries_HOSVD()

# model.calc_logZ()
model.calc_magnetization()
# print(model.magnetization)

  0%|          | 0/15 [00:00<?, ?it/s]

15 0
15


  0%|          | 0/15 [00:00<?, ?it/s]

In [6]:

torch.set_default_tensor_type(torch.cuda.DoubleTensor)

In [7]:
model.magnetization

array(3.5552844e-06, dtype=float32)

In [8]:
from HOTRG import trace_two_tensors
tt=[(trace_two_tensors(T).item(),trace_two_tensors(T_op).item()) for T,T_op in zip(model.Ts,model.T_ops)]
[((b/a if abs(a)>0 else 0),a,b) for a,b in tt]

[(0, 0.0, 0.0),
 (-2.7897324047749574, 0.04779658942265784, -0.13333969435011256),
 (-1.821997652876098, 1.0153369211760674, -1.8499414872612385),
 (1.4392149863065409, 1.192677601453903, 1.716519477844597),
 (1.0142565934901129, 1.030020219631409, 1.0447047991892908),
 (0.8214542841935257, 0.8949524798322831, 0.7351625487078489),
 (0.72428624019466, 1.030275604994425, 0.7462144443056908),
 (0.8787220902480029, 0.9880340700172763, 0.8682073632418226),
 (1.0420737876185537, 0.9505671077898153, 0.990561066400147),
 (1.0852982191023912, 0.9742598610125194, 1.0573624920998306),
 (1.0945176855210959, 0.9899900614161253, 1.083561630710065),
 (1.0951954357030456, 0.993951124144083, 1.088570734474511),
 (1.0952695158500627, 0.9955685004135967, 1.090415829443573),
 (1.0952769848202044, 0.9967958559812493, 1.0917675596204175),
 (1.0952768607987813, 0.9973498644867291, 1.0923742286931146),
 (1.0952768238304655, 0.9974733446743865, 1.0925094368105133)]

In [9]:
from HOTRG import HOTRGLayer
T0=model.get_T0()
T0_op=model.get_SZT0()
layers=[HOTRGLayer(tensor_shape=model.Ts[i].shape,ww=[model.ws[2*i],model.ws[2*i+1]]) for i in range(model.nLayers)]

In [10]:
from HOTRG import forward_observable_tensor
Ts,T_ops,logTotals=forward_observable_tensor(T0,T0_op,layers)

0it [00:00, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 764.00 MiB (GPU 0; 11.17 GiB total capacity; 9.37 GiB already allocated; 524.25 MiB free; 10.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF